In [ ]:
import random

import pandas as pd
import numpy as np
import itertools
import axelrod as axl




In [ ]:
from joblib import Parallel, delayed
import xgboost as xgb


In [ ]:
strategies = [axl.Raider(),axl.Cooperator(), axl.GrudgerAlternator(), axl.EvolvedANNNoise05(), axl.Forgiver()] #----- input of agents


In [ ]:
strategy_order = [player.__class__.__name__ for player in strategies]

strategy_mapping = {name: player.__class__ for name, player in zip(strategy_order, strategies)}

strategy_mapping

In [ ]:
MAX_COUNT = 5          # each strategy appears between 0 and 5 times
NUM_TOTAL_COMPS = (MAX_COUNT+1) ** len(strategy_order)  # 6^5 = 7776
NUM_SIMULATIONS = 5000   # number of tournaments to simulate (sampled from all compositions)
TOURNS = 200           # turns per tournament
REPETITIONS = 10      # repetitions per tournament


def get_tournament_winners_by_score(results, players, use_normalised=True):
    if use_normalised:
        if hasattr(results.normalised_scores, "compute"):
            scores_matrix = np.array(results.normalised_scores.compute())
        else:
            scores_matrix = np.array(results.normalised_scores)
    else:
        if hasattr(results.scores, "compute"):
            scores_matrix = np.array(results.scores.compute())
        else:
            scores_matrix = np.array(results.scores)
    average_scores = [np.mean(scores) for scores in scores_matrix]
    score_dict = {player.__class__.__name__: avg for player, avg in zip(players, average_scores)}
    print("Average scores:", score_dict)
    max_score = max(average_scores)
    winners = [
        player.__class__.__name__ for player, avg in zip(players, average_scores)
        if abs(avg - max_score) < 0.001
    ]
    unique_winners = []
    for w in winners:
        if w not in unique_winners:
            unique_winners.append(w)
    print("Winners for this tournament:", unique_winners, "\n")
    return unique_winners

def run_single_tournament(composition, turns=TOURNS, repetitions=REPETITIONS):
    players = []
    for count, strat_name in zip(composition, strategy_order):
        for _ in range(count):
            players.append(strategy_mapping[strat_name]())
    print("Tournament composition:", composition)
    tourn = axl.Tournament(players, turns=turns, repetitions=repetitions)
    results_tourn = tourn.play()
    coop_matrix = results_tourn.cooperation
    total_cooperations = sum(sum(row) for row in coop_matrix)
    total_moves = len(coop_matrix) ** 2 * repetitions
    coop_rate = total_cooperations / total_moves if total_moves > 0 else 0.0
    winners = get_tournament_winners_by_score(results_tourn, players)
    return {
        "composition": composition,
        "coop_rate": coop_rate,
        "winning_strategy": winners
    }

In [ ]:
all_comps = [tuple(x) for x in itertools.product(range(MAX_COUNT + 1), repeat=len(strategy_order))]
all_comps = [x for x in all_comps if any(x)]

print("Total possible compositions:", len(all_comps))  # Should be 7776

# Randomly sample NUM_SIMULATIONS compositions from the full space.
selected_comps = random.sample(all_comps, NUM_SIMULATIONS)
print("Selected compositions for simulation:", len(selected_comps))


def run_single_tournament_wrapper(comp, idx):
    print(f"Starting tournament simulation {idx+1} of {NUM_SIMULATIONS}")
    return run_single_tournament(list(comp), turns=TOURNS, repetitions=REPETITIONS)

# Run the simulations in parallel.
from joblib import Parallel, delayed
sim_results = Parallel(n_jobs=-1, verbose=10)(
    delayed(run_single_tournament_wrapper)(comp, idx) for idx, comp in enumerate(selected_comps)
)

In [ ]:
df_experiments = pd.DataFrame(sim_results)
